<a href="https://colab.research.google.com/github/sayanbanerjee32/learning-equality-curriculum-recommendations/blob/main/lecr_XLMR_classification_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
is_colab = not os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [2]:
# install fastkaggle if not available
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle
## install collab specific libraries
if is_colab:
    !pip install -Uq datasets
    !pip install -Uq transformers
    !pip install -Uq sentencepiece 
    !pip install -Uq evaluate
    !pip install -Uq pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 67.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.8 MB/s eta 0:00:00


In [3]:
from fastkaggle import *
from pathlib import Path
import pandas as pd
import numpy as np # linear algebra
from tqdm.auto import tqdm
import gc
from functools import partial

import torch
from transformers import (AutoModelForSequenceClassification,
                          AutoTokenizer, 
                          DataCollatorWithPadding, 
                          TrainingArguments,
                          Trainer, 
                          IntervalStrategy)
from datasets import Dataset, DatasetDict, ClassLabel
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
%env TOKENIZERS_PARALLELISM=true

env: TOKENIZERS_PARALLELISM=true


In [24]:
# config depending on whether this is running on kaggle or collab
# is_colab = True
comp = 'learning-equality-curriculum-recommendations'
if is_colab:
    model_save_path = Path('/content/drive/MyDrive/Colab Notebooks/models/'+comp)
    data_save_path = Path('/content/drive/MyDrive/Colab Notebooks/data/'+comp)
else:
    model_save_path = Path('/kaggle/working/'+comp) #+'/models')
    data_save_path = Path('/kaggle/working/'+comp)

In [5]:
# import colab libraries
if is_colab:
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
if is_colab:
    !mkdir ~/.kaggle
    !cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [7]:
# This permissions change avoids a warning on Kaggle tool startup.
if is_colab:
    !chmod 600 ~/.kaggle/kaggle.json

In [8]:
path = setup_comp(comp)
path

100%|██████████| 254M/254M [00:15<00:00, 17.3MB/s]


Path('learning-equality-curriculum-recommendations')

### read all input files

In [9]:
df = pd.read_pickle(data_save_path / 'topic_content_details.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 127832 entries, 0 to 127831
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   topic_id             127832 non-null  object
 1   content_id           127832 non-null  object
 2   label                127832 non-null  int64 
 3   title_topic          127832 non-null  object
 4   description_topic    127832 non-null  object
 5   channel              127832 non-null  object
 6   category             127832 non-null  object
 7   level                127832 non-null  int64 
 8   language_topic       127832 non-null  object
 9   parent               127832 non-null  object
 10  has_content          127832 non-null  bool  
 11  title_content        127832 non-null  object
 12  description_content  127832 non-null  object
 13  kind                 127832 non-null  object
 14  text                 127832 non-null  object
 15  language_content     127832 non-nu

In [10]:
## all has contents - no breadcrums yet
df.has_content.value_counts()

True    127832
Name: has_content, dtype: int64

In [11]:
# pre -process content text
df['masked_content_txt'] = df[['title_content','text']].apply(lambda row: row.text.strip().replace(row.title_content.strip(),
                                                                                                   '__MASKED__').strip().split("\n")[0][:100],
                                                               axis = 1)
df['input'] =  "CONTENT: " + (df["title_content"]
        + " \n "
        + df["description_content"].str.strip()
        + " \n "
        + df["masked_content_txt"]).str.strip() + " TOPIC: " + (df["title_topic"]
        + " \n "
        + df["description_topic"].str.strip()
        ).str.strip()
df['seq_length_input'] = [len(txt.split()) for txt in df['input'].tolist()]
df['seq_length_input'].describe()

count    127832.000000
mean         39.271012
std          61.953274
min           4.000000
25%          11.000000
50%          26.000000
75%          49.000000
max        1520.000000
Name: seq_length_input, dtype: float64

In [12]:
# random sampling to test in collab
if is_colab: df = df.sample(frac=0.10)
df.shape[0]

12783

In [13]:
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['topic_id', 'content_id', 'label', 'title_topic', 'description_topic', 'channel', 'category', 'level', 'language_topic', 'parent', 'has_content', 'title_content', 'description_content', 'kind', 'text', 'language_content', 'copyright_holder', 'license', 'masked_content_txt', 'input', 'seq_length_input', '__index_level_0__'],
    num_rows: 12783
})

In [14]:
# label encoding
labels = ClassLabel(names=df.label.unique().tolist())

In [15]:
model_nm = 'xlm-roberta-base'
# 'bert-base-multilingual-cased'

In [16]:
tokz = AutoTokenizer.from_pretrained(model_nm)
tokz.model_max_length, tokz.is_fast

(512, True)

In [17]:
def tok_func(batch, is_test = False): 
    tokens = tokz(batch["input"], padding="longest", truncation=True)
    if not is_test:
        tokens['labels'] = labels.str2int(batch['label'])
        # [float(l) for l in labels.str2int(batch['discourse_effectiveness'])]
    return tokens
tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/12783 [00:00<?, ? examples/s]

In [18]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['topic_id', 'content_id', 'label', 'title_topic', 'description_topic', 'channel', 'category', 'level', 'language_topic', 'parent', 'has_content', 'title_content', 'description_content', 'kind', 'text', 'language_content', 'copyright_holder', 'license', 'masked_content_txt', 'input', 'seq_length_input', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 9587
    })
    test: Dataset({
        features: ['topic_id', 'content_id', 'label', 'title_topic', 'description_topic', 'channel', 'category', 'level', 'language_topic', 'parent', 'has_content', 'title_content', 'description_content', 'kind', 'text', 'language_content', 'copyright_holder', 'license', 'masked_content_txt', 'input', 'seq_length_input', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 3196
    })
})

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokz)

In [20]:
bs = 16
grad_acc = 16
epochs = 4
lr = 5e-5
metric_name = "f1"

args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=True,
                        evaluation_strategy="epoch",
                         logging_strategy = "epoch",
                        #  logging_steps = 10,
                        #  eval_steps = 10,
                         per_device_train_batch_size=bs,
                          per_device_eval_batch_size=bs*2,
                        num_train_epochs=epochs,
                          weight_decay=0.01,
                         gradient_accumulation_steps=grad_acc,
                        #   load_best_model_at_end=True,
                            metric_for_best_model=metric_name,
                        #  label_names = labels.names, # possibly this is messing up with metrics
                          report_to='none')

In [21]:
# helper function cleaning GPU memory
def report_gpu():
    if is_colab: print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()
report_gpu()

GPU:0
no processes are running


In [22]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, 
                                                           num_labels=labels.num_classes)
def acc_metrics(eval_preds):
    metric = evaluate.load("f1")
    # metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions,
                          references=labels,
                          average = 'weighted')
    # return {"accuracy": (predictions == labels).astype(np.float32).mean().item()}

trainer = Trainer(model, args, 
                  train_dataset=dds['train'], 
                  eval_dataset=dds['test'],
                  data_collator=data_collator,
                  tokenizer=tokz,
                  compute_metrics=acc_metrics)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [23]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: license, description_content, __index_level_0__, title_content, text, content_id, description_topic, has_content, seq_length_input, kind, topic_id, language_content, category, parent, channel, level, title_topic, copyright_holder, language_topic, masked_content_txt, input. If license, description_content, __index_level_0__, title_content, text, content_id, description_topic, has_content, seq_length_input, kind, topic_id, language_content, category, parent, channel, level, title_topic, copyright_holder, language_topic, masked_content_txt, input are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implemen

Epoch,Training Loss,Validation Loss,F1
0,0.587400,0.489048,0.699110
1,0.490800,0.501200,0.747636
2,0.434400,0.421229,0.800230
3,0.387900,0.421989,0.801843


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: license, description_content, __index_level_0__, title_content, text, content_id, description_topic, has_content, seq_length_input, kind, topic_id, language_content, category, parent, channel, level, title_topic, copyright_holder, language_topic, masked_content_txt, input. If license, description_content, __index_level_0__, title_content, text, content_id, description_topic, has_content, seq_length_input, kind, topic_id, language_content, category, parent, channel, level, title_topic, copyright_holder, language_topic, masked_content_txt, input are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3196
  Batch size = 32


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: license, description_content, __index_level_0__, title_content, text, content_id, description_topic, has_content, seq_length_input, kind, topic_id, language_content, category, parent, channel, level, title_topic, copyright_holder, language_topic, masked_content_txt, input. If license, description_content, __index_level_0__, title_content, text, content_id, description_topic, has_content, seq_length_input, kind, topic_id, language_content, category, parent, channel, level, title_topic, copyright_holder, language_topic, masked_content_txt, input are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3196
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.for

In [25]:
### dump model
model_save_path.mkdir(parents=True, exist_ok=True)
trainer.save_model(model_save_path)

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/models/learning-equality-curriculum-recommendations
Configuration saved in /content/drive/MyDrive/Colab Notebooks/models/learning-equality-curriculum-recommendations/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/models/learning-equality-curriculum-recommendations/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/models/learning-equality-curriculum-recommendations/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/models/learning-equality-curriculum-recommendations/special_tokens_map.json


In [28]:
if not iskaggle:
    push_notebook('saan', comp,
                  title='LECR: XLMR - classification- baseline',
                  file='/content/drive/MyDrive/Colab Notebooks/lecr_XLMR_classification_baseline.ipynb',
                  competition=comp, private=False, gpu=True)

Your kernel title does not resolve to the specified id. This may result in surprising behavior. We suggest making your title something that resolves to the specified id. See https://en.wikipedia.org/wiki/Clean_URL#Slug for more information on how slugs are determined.
Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/saansd2003/lecr-xlmr-classification-baseline
